In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Final Preprocessing - Workbook, worksheet to total_solve_df

In [ ]:
# 채점 결과가 있는 데이터만 사용 
worksheet = worksheet[(worksheet['result'] == 'CORRECT') | (worksheet['result'] == 'WRONG')]
workbook = workbook[(workbook['result'] == 'CORRECT') | (workbook['result'] == 'WRONG')]

In [ ]:
# workbook groupby

workbook_groupby = (
    pd.DataFrame(
        workbook.groupby(['student_id', 'update_datetime', 'problem_level', 'result'])['workbook_problem_id'].count()
    )
    .pivot_table(
        index=['student_id', 'update_datetime', 'problem_level'], columns='result', values='workbook_problem_id'
    ).fillna(0).reset_index()
    .assign(
        score=lambda x: x['CORRECT'] / (x['CORRECT'] + x['WRONG']) * x['problem_level']
    )
    .assign(
        cumulative_correct = lambda x: x.groupby(['student_id','problem_level'])['CORRECT'].cumsum()
    )
    .assign(
        cumulative_wrong = lambda x: x.groupby(['student_id','problem_level'])['WRONG'].cumsum()
    )
    .assign(
        cumulative_score=lambda x: x['cumulative_correct'] / (x['cumulative_wrong'] + x['WRONG']) * x['problem_level']
    )
    .assign(
        classification_type = 'workbook'
    )
)


# worksheet groupby

worksheet_groupby = (
    pd.DataFrame(
        worksheet.groupby(['student_id', 'update_datetime', 'problem_level', 'result'])['worksheet_problem_id'].count()
    )
    .pivot_table(
        index=['student_id', 'update_datetime', 'problem_level'], columns='result', values='worksheet_problem_id'
    ).fillna(0).reset_index()
    .assign(
        score=lambda x: x['CORRECT'] / (x['CORRECT'] + x['WRONG']) * x['problem_level']
    )
    .assign(
        cumulative_correct = lambda x: x.groupby(['student_id','problem_level'])['CORRECT'].cumsum()
    )
    .assign(
        cumulative_wrong = lambda x: x.groupby(['student_id','problem_level'])['WRONG'].cumsum()
    )
    .assign(
        cumulative_score=lambda x: x['cumulative_correct'] / (x['cumulative_wrong'] + x['WRONG']) * x['problem_level']
    )
    .assign(
        classification_type = 'worksheet'
    )
)

# Final Preprocessing - Cumulative Scoring Pivot table

In [ ]:
total_solve_groupby = (
      pd.DataFrame(
      total_solve_df.groupby(['academy_create_datetime', 'academy_id','academy_status',
                        'student_id','student_school_type', 'student_status','student_grade', 
                        'date','year', 'month', 'quarter', 
                        'grade','problem_level','result' , 'classification_type'])['problem_id'].count()
      )
        .pivot_table(
            index=['academy_create_datetime', 'academy_id','academy_status',
                   'student_id','student_school_type','student_status', 'student_grade', 
                   'date','year', 'month', 'quarter','grade', 'problem_level', 'classification_type'], columns='result', values='problem_id'
            ).fillna(0).reset_index()
            .assign(
                quarter_dummy = lambda x: x['year']**2 + x['quarter']
            )             
            .assign(
                score=lambda x: x['CORRECT'] / (x['CORRECT'] + x['WRONG']) * x['problem_level']
            )
            .assign(
                solve_num =lambda x: x['CORRECT'] + x['WRONG']
            )            
            .assign(
                cumulative_correct=lambda x: x.groupby(['student_id', 'problem_level'])['CORRECT'].cumsum()
            )
            .assign(
                cumulative_wrong=lambda x: x.groupby(['student_id', 'problem_level'])['WRONG'].cumsum()
            )
            .assign(
                cumulative_score=lambda x: x['cumulative_correct'] / (x['cumulative_correct'] + x['cumulative_wrong']) * x['problem_level']
            ) #연도별 누적
            .assign(
                year_cumulative_correct=lambda x: x.groupby(['student_id', 'problem_level', 'year'])['CORRECT'].cumsum()
            )
            .assign(
                year_cumulative_wrong=lambda x: x.groupby(['student_id', 'problem_level', 'year'])['WRONG'].cumsum()
            )
            .assign(
                year_solve_num =lambda x: x['year_cumulative_correct'] + x['year_cumulative_wrong']
            )             
            .assign(
                 year_cumulative_score=lambda x: x['year_cumulative_correct'] / (x['year_cumulative_correct'] + x['year_cumulative_wrong']) * x['problem_level']
            ) #연도_분기별 누적
            .assign(
                quarter_cumulative_correct=lambda x: x.groupby(['student_id', 'problem_level', 'year', 'quarter'])['CORRECT'].cumsum()
            )
            .assign(
                quarter_cumulative_wrong=lambda x: x.groupby(['student_id', 'problem_level', 'year', 'quarter'])['WRONG'].cumsum()
            )
            .assign(
                quarter_solve_num =lambda x: x['quarter_cumulative_correct'] + x['quarter_cumulative_wrong']
            )             
            .assign(
                quarter_cumulative_score=lambda x: x['quarter_cumulative_correct'] / (x['quarter_cumulative_correct'] + x['quarter_cumulative_wrong']) * x['problem_level']
            ) #전체기간_과목별 누적
            .assign(
                grade_cumulative_correct=lambda x: x.groupby(['student_id', 'problem_level', 'grade'])['CORRECT'].cumsum()
            )
            .assign(
                grade_cumulative_wrong=lambda x: x.groupby(['student_id', 'problem_level', 'grade'])['WRONG'].cumsum()
            )
            .assign(
                grade_solve_num =lambda x: x['grade_cumulative_correct'] + x['grade_cumulative_wrong']
            )             
            .assign(
                grade_cumulative_score=lambda x: x['grade_cumulative_correct'] / (x['grade_cumulative_correct'] + x['grade_cumulative_wrong']) * x['problem_level']
            ) # material별 계산
            .assign(
                score=lambda x: x['CORRECT'] / (x['CORRECT'] + x['WRONG']) * x['problem_level']
            )
            .assign(
                solve_num =lambda x: x['CORRECT'] + x['WRONG']          
            )#전체기간_material별 누적
            .assign(
                material_cumulative_correct=lambda x: x.groupby(['student_id', 'problem_level', 'classification_type'])['CORRECT'].cumsum()
            )
            .assign(
                material_cumulative_wrong=lambda x: x.groupby(['student_id', 'problem_level', 'classification_type'])['WRONG'].cumsum()
            )
            .assign(
                material_solve_num =lambda x: x['material_cumulative_correct'] + x['material_cumulative_wrong']
            )             
            .assign(
                material_cumulative_score=lambda x: x['material_cumulative_correct'] / (x['material_cumulative_correct'] + x['material_cumulative_wrong']) * x['problem_level']
            ) # year_material별 누적
            .assign(
                year_material_cumulative_correct=lambda x: x.groupby(['student_id', 'problem_level', 'year', 'classification_type'])['CORRECT'].cumsum()
            )
            .assign(
                year_material_cumulative_wrong=lambda x: x.groupby(['student_id', 'problem_level', 'year', 'classification_type'])['WRONG'].cumsum()
            )
            .assign(
                year_material_solve_num =lambda x: x['year_material_cumulative_correct'] + x['year_material_cumulative_wrong']
            )             
            .assign(
                year_material_cumulative_score=lambda x: x['year_material_cumulative_correct'] / (x['year_material_cumulative_correct'] + x['year_material_cumulative_wrong']) * x['problem_level']
            )#quarter_material별 누적
            .assign(
                quarter_material_cumulative_correct=lambda x: x.groupby(['student_id', 'problem_level', 'year','quarter', 'classification_type'])['CORRECT'].cumsum()
            )
            .assign(
                quarter_material_cumulative_wrong=lambda x: x.groupby(['student_id', 'problem_level', 'year','quarter','classification_type'])['WRONG'].cumsum()
            )
            .assign(
                quarter_material_solve_num =lambda x: x['quarter_material_cumulative_correct'] + x['quarter_material_cumulative_wrong']
            )             
            .assign(
                quarter_material_cumulative_score=lambda x: x['quarter_material_cumulative_correct'] / (x['quarter_material_cumulative_correct'] + x['quarter_material_cumulative_wrong']) * x['problem_level']
            )
            .groupby(['academy_create_datetime', 'academy_id', 'academy_status', 
             'student_id','student_school_type', 'student_status', 'student_grade', 
             'grade','date','year', 'month', 'quarter','quarter_dummy', 'classification_type'])['solve_num','score','cumulative_score', 
                                                         'year_solve_num','year_cumulative_score', 
                                                         'quarter_solve_num','quarter_cumulative_score',
                                                          'grade_solve_num','grade_cumulative_score',
                                                          'material_solve_num', 'material_cumulative_score',
                                                          'year_material_solve_num', 'year_material_cumulative_score',
                                                          'quarter_material_solve_num', 'quarter_material_cumulative_score'].sum().reset_index()
                                                                      .assign(
                                                                          day = lambda x: int(x['date'].to_string()[-2:])/8
                                                                          )
                                                                      .assign(
                                                                          day_cum_sum = lambda x: x.groupby(['student_id'])['day'].cumsum()
                                                                          ) 
​
​
    )
# 최초 quarter 여부 태깅 
quarter_dummy_df = pd.DataFrame(total_solve_groupby.groupby(['student_id'])['quarter_dummy'].min())
quarter_dummy_df.columns = ['quarter_dummy_2']
​
total_solve_groupby = pd.merge(total_solve_groupby, quarter_dummy_df, 
                           left_on='student_id', 
                           right_on='student_id', 
                           how='left')         
total_solve_groupby['first_quarter_tag'] = (total_solve_groupby['quarter_dummy'] == total_solve_groupby['quarter_dummy_2'])     
total_solve_groupby = total_solve_groupby[total_solve_groupby.columns.difference(['quarter_dummy','quarter_dummy_2'])]
​
​
​
​
# 전체 최종 일자
max_date_dummy_df = pd.DataFrame(total_solve_groupby.groupby(['student_id'])['day_cum_sum'].max())
max_date_dummy_df.columns = ['day_cum_sum_2']
total_solve_groupby = pd.merge(total_solve_groupby, max_date_dummy_df, 
                           left_on='student_id', 
                           right_on='student_id', 
                           how='left') 
total_solve_groupby['final_date_tag'] = (total_solve_groupby['day_cum_sum'] == total_solve_groupby['day_cum_sum_2'])     
total_solve_groupby = total_solve_groupby[total_solve_groupby.columns.difference(['day_cum_sum_2'])]
​
​
#연도별 최종 일자
year_max_date_dummy_df = pd.DataFrame(total_solve_groupby.groupby(['student_id','year'])['day_cum_sum'].max())
year_max_date_dummy_df.columns = ['day_cum_sum_2']
total_solve_groupby = pd.merge(total_solve_groupby, year_max_date_dummy_df, 
                           left_on=['student_id', 'year'],
                           right_on=['student_id', 'year'], 
                           how='left') 
total_solve_groupby['year_final_date_tag'] = (total_solve_groupby['day_cum_sum'] == total_solve_groupby['day_cum_sum_2'])     
total_solve_groupby = total_solve_groupby[total_solve_groupby.columns.difference(['day_cum_sum_2'])]
​
​
#quarter별 최종 일자
quarter_max_date_dummy_df = pd.DataFrame(total_solve_groupby.groupby(['student_id','year','quarter'])['day_cum_sum'].max())
quarter_max_date_dummy_df.columns = ['day_cum_sum_2']
total_solve_groupby = pd.merge(total_solve_groupby, quarter_max_date_dummy_df, 
                           left_on=['student_id', 'year','quarter'],
                           right_on=['student_id', 'year', 'quarter'],
                           how='left') 
total_solve_groupby['quarter_final_date_tag'] = (total_solve_groupby['day_cum_sum'] == total_solve_groupby['day_cum_sum_2'])     
total_solve_groupby = total_solve_groupby[total_solve_groupby.columns.difference(['day_cum_sum_2'])]
​
​
#최초 quarter 마지막 일자 등급 태깅
from sklearn.preprocessing import StandardScaler
scaler_ = StandardScaler()
​
scaler_.fit( pd.DataFrame(total_solve_groupby['quarter_cumulative_score']) )
total_solve_groupby['scaled_score'] = scaler_.transform(pd.DataFrame(total_solve_groupby['quarter_cumulative_score'] ))
​
total_solve_groupby = total_solve_groupby.assign(
    dummy_first_quarter_score_rating = lambda x: x['scaled_score'].apply(rating)
)
​
​
first_quarter_score_rating_df = (total_solve_groupby[ 
    (total_solve_groupby['first_quarter_tag'] == True) 
    & (total_solve_groupby['quarter_final_date_tag'] == True) ]
      [['student_id','dummy_first_quarter_score_rating']]
)
first_quarter_score_rating_df.columns = ['student_id', 'first_quarter_score_rating']
total_solve_groupby = pd.merge(total_solve_groupby, first_quarter_score_rating_df, 
                           left_on='student_id',
                           right_on='student_id',
                           how='left') 
total_solve_groupby = total_solve_groupby[total_solve_groupby.columns.difference(['dummy_first_quarter_score_rating'])]
​
​
# 학원 내 학생들 평균 등급 / 학원 등급 태깅 
academy_rating_df = (
     pd.DataFrame(     
        pd.DataFrame(
        total_solve_groupby.groupby(['academy_id','student_id'])['first_quarter_score_rating'].mean()
        ).reset_index()
        .groupby(['academy_id'])['first_quarter_score_rating'].mean()
    )
  )
academy_rating_df.columns = ['mean_student_rating_of_academy']
​
​
from sklearn.preprocessing import StandardScaler
scaler_ = StandardScaler()
​
scaler_.fit( pd.DataFrame(academy_rating_df['mean_student_rating_of_academy']) )
academy_rating_df['scaled_rating'] = scaler_.transform(pd.DataFrame(academy_rating_df['mean_student_rating_of_academy'] ))
​
academy_rating_df = (
    academy_rating_df.assign(
        academy_rating = lambda x: x['scaled_rating'].apply(rating)
        ).reset_index()
        [['academy_id', 'mean_student_rating_of_academy', 'scaled_rating','academy_rating']]
)
total_solve_groupby = pd.merge(total_solve_groupby, academy_rating_df, 
                           left_on='academy_id',
                           right_on='academy_id',
                           how='left') 
​
​
total_solve_groupby